# NSRR MESA Features extraction

https://sleepdata.org/datasets/mesa/pages/equipment/montage-and-sampling-rate-information.md

In [ ]:
import os
import yasa
import warnings
import numpy as np
import pandas as pd
import pingouin as pg
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
from scipy.signal import hilbert
from mne.filter import filter_data
from tensorpac import methods as tpm

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.25)

# Define paths
root_dir = '/Volumes/NSRR/mesa/'
eeg_dir = root_dir + 'polysomnography/edfs/'
hypno_dir = root_dir + 'polysomnography/annotations-events-profusion/'

# List all EDF files
all_edfs = os.listdir(eeg_dir)
all_subj = [c.split("-")[2][:-4] for c in all_edfs]

print(len(all_subj))

In [ ]:
df = []
dic_erpac = {}
include = ['EEG3']  # EEG3 is C4-M1
sf = 100

np.seterr(all="ignore")

for sub in tqdm(all_subj):
    
    #################################
    # DATA LOADING
    #################################
        
    eeg_file = eeg_dir + 'mesa-sleep-' + str(sub) + '.edf'
    hypno_file = hypno_dir + 'mesa-sleep-' + str(sub) + '-profusion.xml'
    
    # Check that file exists
    if not os.path.isfile(eeg_file):
        warnings.warn("File not found %s" % eeg_file)
        continue
    if not os.path.isfile(hypno_file):
        warnings.warn("File not found %s" % hypno_file)
        continue

    # LOAD EEG DATA
    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw.drop_channels(np.setdiff1d(raw.info['ch_names'], include))
        raw.load_data()
    except:
        continue
        
    # Resample to 100 Hz and low-pass filter 
    raw.resample(sf, npad="auto")

    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # Check that hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]

    # Set NREM is 6
    hypno_NREM = pd.Series(hypno).replace({2: 6, 3: 6}).to_numpy()
        
    # Upsample hypno NREM
    hypno_NREM_up = yasa.hypno_upsample_to_data(hypno_NREM, sf_hyp, raw)
    
    # Extract C4-M1 data and inverse polarity
    # http://zzz.bwh.harvard.edu/luna/vignettes/nsrr-polarity/
    data = (raw.get_data()[0]) * 1e6
    data *= -1 
    
    #################################
    # PHASE-AMPLITUDE COUPLING
    #################################
    
    # Slow-waves detection in NREM (N2 + N3)
    # Same parameters as WASHU
    ptp_thr = 60
    neg_amp = round(ptp_thr / 1.875)
    freq_sw = (0.3, 1.5)
    freq_sp = (12, 16)
    sw = yasa.sw_detect(
            data, sf, ch_names=["C4"], hypno=hypno_NREM_up, include=6, freq_sw=freq_sw,
            dur_neg=(0.3, 1.5), dur_pos=(0.1, 1), amp_neg=(neg_amp, 200),
            amp_pos=(10, 200), amp_ptp=(ptp_thr, 300), coupling=False,
            freq_sp=freq_sp, remove_outliers=True
    )
    
    if sw is None:
        warnings.warn("No SOs detected for %s. SKIPPING SUBJECT." % sub)
        continue
        
    # Spindles detection in NREM (N2 + N3) -- same parameters as WashU
    sp = yasa.spindles_detect(
            data, sf, ch_names=["C4"], hypno=hypno_NREM_up, include=6, duration=(0.4, 2), 
            thresh={'rel_pow': 0.05, 'corr': 0.50, 'rms': 2}, remove_outliers=True)
    
    if sp is None:
        warnings.warn("No spindles detected for %s. SKIPPING SUBJECT." % sub)
        continue
        
    sp_sum = sp.summary(grp_stage=True)
        
    # Co-occurring slow-waves / spindles and get summary
    sw_sum = sw.find_cooccurring_spindles(sp.summary())
    sw_sum = sw.summary(grp_chan=False, grp_stage=True)
    
    # Assert that we have at least 5 slow-waves, otherwise skip subject
    if sw._events.shape[0] <= 5:
        warnings.warn("Less than 5 SOs detected for %s. SKIPPING SUBJECT." % sub)
        continue
    
    # PAC using only +/- 1 second around negative peak, as in Mikutta et al 2021 (J. Sleep Research)
    data_sp = filter_data(
        sw._data, sf, 12, 16, method='fir', l_trans_bandwidth=1.5,
        h_trans_bandwidth=1.5, verbose=0)
    amp = np.squeeze(np.abs(hilbert(data_sp)))
    pha = np.squeeze(np.angle(hilbert(sw._data_filt)))
    
    # Get epochs
    idx_pks = (sw._events['NegPeak'] * sf).to_numpy().astype(int)  # In samples
    idx_epochs = yasa.get_centered_indices(amp, idx_pks, npts_before=sf, npts_after=sf)[0]
    
    amp = amp[idx_epochs][None, ...]  # shape (n_freqs, n_epochs, n_samples)
    pha = pha[idx_epochs][None, ...]
    
    # Calculate raw PAC and preferred phase
    ndp = np.squeeze(tpm.norm_direct_pac(pha, amp, p=1))
    ndp_thr = np.squeeze(tpm.norm_direct_pac(pha, amp))  # Unreliable values are set to 0
    idx_thr_supzero = np.nonzero(ndp_thr)
    ndp_thr_supzero = ndp_thr[idx_thr_supzero]
    pp = np.squeeze(tpm.preferred_phase(pha, amp, n_bins=18)[1])
    pp_thr_supzero = pp[idx_thr_supzero]
    
    # Append to output dict
    output_dict = {
        "dataset": "mesa",
        "subj": sub,
        # Spindles
        "sp_density": sp_sum.loc[6, "Density"],
        "sp_power": sp_sum.loc[6, "RelPower"],
        "sp_freq": sp_sum.loc[6, "Frequency"],
        # Slow-waves
        "sw_density": sw_sum.loc[6, "Density"],
        "sw_ptp": sw_sum.loc[6, "PTP"],
        "sw_freq": sw_sum.loc[6, "Frequency"],
        "sw_ndpac": np.nanmean(ndp),
        # "sw_ndpac_thr": np.nanmean(ndp_thr),
        "sw_ndpac_thr_supzero": np.nanmean(ndp_thr_supzero),
        "sw_ndpac_prop_supzero": ndp_thr_supzero.size / ndp_thr.size,
        "sw_pp": pg.circ_mean(pp),
        "sw_pp_thr_supzero": pg.circ_mean(pp_thr_supzero),
        "sw_coinc_spindles": sw_sum.loc[6, "CooccurringSpindle"],
    }

    # Append to main dataframe
    df.append(output_dict)

In [ ]:
df = pd.DataFrame(df).set_index(["dataset", "subj"])
df.round(3)

In [ ]:
df.to_csv("../output/csv/df_mesa_coupling_NREM_inverted.csv")